In [2]:
from helpers import extract_code, get_dataset


dataset = get_apps_dataset()

Filter: 100%|██████████| 5000/5000 [00:00<00:00, 11992.94 examples/s]


In [17]:
system_message = """You are an expert Python programmer. You will be given a question (problem specification) and will generate a
correct Python program that matches the specification and passes all tests. You will NOT return
anything except for the program. Put your fixed program within code delimiters, for example: ```python```. Do not call the function, just define it."""
user_message = sample['question']

In [18]:
def CodeModel():
    return llm

from langchain.schema import SystemMessage, HumanMessage
from langchain.callbacks import get_openai_callback

messages = [
    SystemMessage(
        content=system_message,
    ),
    HumanMessage(
        content=user_message,
    ),
]

with get_openai_callback() as cb:
    response = llm(messages)
    print(cb)


Tokens Used: 813
	Prompt Tokens: 621
	Completion Tokens: 192
Successful Requests: 1
Total Cost (USD): $0.03015


In [21]:
code = extract_code(response.content)

print(code)
# exec(code, globals())



def minimum_D(n, a):
    a = sorted(a)
    if (a[-1] - a[0]) % 2 == 0:
        D = (a[-1] - a[0]) // 2
        for i in range(n):
            if a[i] < a[0] + D:
                a[i] += D
            elif a[i] > a[0] + D:
                a[i] -= D
        if len(set(a)) == 1:
            return D
    D = a[-1] - a[0]
    for i in range(n):
        if a[i] < a[0] + D:
            a[i] += D
        elif a[i] > a[0] + D:
            a[i] -= D
    if len(set(a)) == 1:
        return D
    return -1

